# :snowflake: Snowflake Notebooks へようこそ :notebook:

[Snowflake Notebooks](https://docs.snowflake.com/LIMITEDACCESS/snowsight-notebooks/ui-snowsight-notebooks-about)でPythonとSQLをシームレスに活用して、データ分析を次のレベルに引き上げましょう！⚡️

初めてのプロジェクトを始めるためのクイックノートブックです！🚀

## Pythonパッケージの追加 🎒

ノートブックには、numpy、pandas、matplotlibなど、データサイエンス🧪や機械学習🧠でよく使われるPythonライブラリが事前にインストールされています！

他のパッケージを使用したい場合は、右上の`Packages`ドロップダウンをクリックして、ノートブックに追加のパッケージを追加してください。

このデモでは、ノートブック作成時にenvironment.ymlの一部として`matplotlib`と`scipy`パッケージを追加します。

In [ ]:
# このノートブックで使用するPythonパッケージをインポート
import streamlit as st
import altair as alt

# ノートブックに事前インストールされているライブラリ
import pandas as pd
import numpy as np

# 追加したパッケージ
import matplotlib.pyplot as plt

## ローカルパッケージの追加
ノートブックにはファイルを追加する機能があり、pythonスクリプトを追加することも可能になっています。

サイドバーで `ファイル` タブを開き、`+`アイコンをクリックするとファイルのアップロード・追加が行えます。

> **注意**
> ノートブックでPythonパッケージとして使用するファイルをアップロードする場合、Warehouse Runtimeで実行する際は
> `.py`と `.zip` ファイルのみがサポートされている点にご注意ください。
> Container Runtimeでは、.whl（wheel）ファイルもサポートされています。 `.zip` ファイルとしてパッケージをインポートする場合、
> ルートディレクトリに `__init__.py `ファイルが存在し、それがPythonパッケージであることを示す必要があります。

In [ ]:
from getCustomerFakerData import gen_fake_customers

df_fake_customers = gen_fake_customers(100)
df_fake_customers

## プライベートリポジトリからのインポート
```
コンテナランタイムでのみ使用可能
```
Pipは、JFrog Artifactoryのようなプライベートソースからのパッケージのインストールを基本認証でサポートしています。ノートブックを外部アクセス統合（External Access Integration）用に設定し、リポジトリにアクセスできるようにします。

1. ネットワークルールを作成し、アクセスしたいリポジトリを指定します。例えば、このネットワークルールはJFrogリポジトリを指定しています:
```sql
CREATE OR REPLACE NETWORK RULE jfrog_network_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('<your-repo>.jfrog.io');
```
2. 外部ネットワーク位置への認証に必要な資格情報を表すシークレットを作成します。
```sql
CREATE OR REPLACE SECRET jfrog_token
  TYPE = GENERIC_STRING
  SECRET_STRING = '<your-jfrog-token>';
```
3. リポジトリへのアクセスを許可する外部アクセス統合を作成します：
```sql
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION jfrog_integration
  ALLOWED_NETWORK_RULES = (jfrog_network_rule)
  ALLOWED_AUTHENTICATION_SECRETS = (jfrog_token)
  ENABLED = TRUE;

GRANT USAGE ON INTEGRATION jfrog_integration TO ROLE data_scientist;
```
4. 外部アクセス統合とシークレットをノートブックに関連付けます。
```sql
ALTER NOTEBOOK my_notebook
  SET EXTERNAL_ACCESS_INTEGRATIONS = (jfrog_integration),
    SECRETS = ('jfrog_token' = jfrog_token);
```
5. 外部アクセス設定にアクセスするには、ノートブックの右上にある「ワークシートのその他のアクション」（ノートブックアクションメニュー）を選択します。
6. 「ノートブック設定」を選択し、次に「External Access」タブを選択します。
7. リポジトリに接続する外部アクセス統合を選択します。
ノートブックが再起動します。
8. ノートブックが再起動されたら、リポジトリからインストール可能になります
```cmd
!pip install hello-jfrog --index-url https://<user>:<token>@<your-repo>.jfrog.io/artifactory/api/pypi/test-pypi/simple
```

## 手軽なSQLクエリ 💡 

同じワークシート内でPythonとSQLを簡単に切り替えることができます。

サンプルデータを生成するSQLを書いてみましょう。 

In [ ]:
-- スノーボード製品の合成データセットを価格と評価と共に生成
SELECT CONCAT('SNOW-',UNIFORM(1000,9999, RANDOM())) AS PRODUCT_ID, 
       ABS(NORMAL(5, 3, RANDOM())) AS RATING, 
       ABS(NORMAL(750, 200::FLOAT, RANDOM())) AS PRICE
FROM TABLE(GENERATOR(ROWCOUNT => 100));

## Pythonでの作業に戻る 🐍

セルに名前を付けて、後続のセルでその出力を参照することができます。

SQL結果にPythonから直接アクセスし、結果をpandas DataFrameに変換できます。🐼

```python
# SQLセルの出力をSnowpark DataFrameとしてアクセス
my_snowpark_df = sql_querying.to_df()
``` 

```python
# SQLセルの出力をpandas DataFrameに変換
my_df = sql_querying.to_pandas()
``` 

In [ ]:
df = sql_querying.to_pandas()
df

## 📊 データの可視化

[Altair](https://altair-viz.github.io/)を使用して、データ分布をヒストグラムとして簡単に可視化できます。

In [ ]:
# Altairで結果をプロット
chart = alt.Chart(df,title="評価分布").mark_bar().encode(
    alt.X("RATING", bin=alt.Bin(step=2)),
    y='count()',
)

st.altair_chart(chart)

チャートをカスタマイズして、カーネル密度推定（KDE）と中央値をプロットしたいとします。matplotlibを使用して価格分布をプロットできます。`.plot`コマンドは内部的に`scipy`を使用してKDEプロファイルを計算することに注意してください。これは、このチュートリアルの前半でパッケージとして追加したものです。

In [ ]:
fig, ax = plt.subplots(figsize = (6,3))
plt.tick_params(left = False, right = False , labelleft = False) 

price = df["PRICE"]
price.plot(kind = "hist", density = True, bins = 15)
price.plot(kind="kde", color='#c44e52')


# パーセンタイルを計算
median = price.median()
ax.axvline(median,0, color='#dd8452', ls='--')
ax.text(median,0.8, f'中央値: {median:.2f}  ',
        ha='right', va='center', color='#dd8452', transform=ax.get_xaxis_transform())

# チャートを美しくする
plt.style.use("bmh")
plt.title("価格分布")
plt.xlabel("価格 (ビン化)")
left, right = plt.xlim()   
plt.xlim((0, right))  
# 目盛りと軸線を削除
ax.tick_params(left = False, bottom = False)
for ax, spine in ax.spines.items():
    spine.set_visible(False)

plt.show()

## Snowparkを使用したデータ操作 🛠️

お気に入りのPythonデータサイエンスライブラリを使用することに加えて、[Snowpark API](https://docs.snowflake.com/en/developer-guide/snowpark/index)を使用してノートブック内でデータをスケールでクエリおよび処理することもできます。

まず、アクティブなノートブックセッションを通じてsession変数を直接取得できます。session変数は、SnowflakeのPython APIを使用するためのアクセスポイントです。

In [ ]:
import streamlit as st

st.image("images/Snowpark.png")

Snowflakeの `QUERY_TAG` は、セッションレベルでオプションとして設定可能なパラメーターであり、ユーザーがセッション内で実行される任意のSQL文に文字列を関連付けることができます。このタグは、`QUERY_HISTORY` ビューおよびテーブル関数に記録され、クエリ活動の監視、監査、分析に役立つ重要なメカニズムを提供します。[クエリ履歴でクエリのアクティビティをモニターする](https://docs.snowflake.com/ja/user-guide/ui-snowsight-activity#review-query-history-in-snowsight)

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()
# セッションにクエリタグを追加。デバッグとパフォーマンス監視に役立ちます。
session.query_tag = {"origin":"sf_sit-is", "name":"notebook_demo_pack", "version":{"major":1, "minor":0}, "attributes":{"is_quickstart":0, "source":"notebook"}}

例えば、Snowparkを使用してpandas DataFrameをSnowflakeのテーブルに保存することができます。💾

In [ ]:
session.write_pandas(df,"SNOW_CATALOG",auto_create_table=True, table_type="temp")

`SNOW_CATALOG`テーブルが作成されたので、以下の構文を使用してテーブルをロードできます：

```python
df = session.table("<DATABASE_NAME>.<SCHEMA_NAME>.<TABLE_NAME>")
```

セッションがアクセスしたいテーブルのデータベースとスキーマに既に設定されている場合は、テーブル名を直接参照できます。

In [ ]:
df = session.table("SNOW_CATALOG")

テーブルをロードしたら、Snowparkの[`describe`](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.DataFrame.describe)を呼び出して基本的な記述統計を計算できます。 

In [ ]:
df.describe()

## Pandas on Snowflake

[pandas on Snowflake](https://docs.snowflake.com/ja/developer-guide/snowpark/python/pandas-on-snowflake)は、開発者がSnowflake内のデータに対して直接pandasコードを実行できるようにします。ユーザーは、Snowflakeのパフォーマンス、スケーラビリティ、ガバナンスを活用しながら、慣れ親しんだpandasのネイティブ体験を同じように得ることができます。

このクイックスタートでは、Snowpark pandas API を使用して Snowflake で pandas を実行する方法を示します。また、Snowpark pandas API がネイティブの pandas API と非常に類似しており、従来の pandas パイプラインを数行のコード変更だけでスケールアップできることも確認できます。このノートブックは Snowflake Notebook で実行できます。

### pandas on Snowflakeを使うべき時
以下のいずれかに当てはまる場合は、pandas on Snowflakeを使用する必要があります。

- あなたはpandas API と、より広い PyData エコシステムに精通しています。
- pandasに精通し、同じコードベースで共同作業をしたい人とチームで仕事をします。
- pandasで書かれた既存のコードがあります
- あなたのワークフローには、pandas DataFrames でサポートされているように、注文に関連するニーズがあります。例えば、ワークフロー全体でデータセットが同じ順序で並んでいる必要があります
- AI-ベースのコパイロット・ツールによる、より正確なコード補完を好みます。

**注意**) 以下のサンプルコードは、Snowflake Marketplaceにある[Finance & Economics](https://app.snowflake.com/marketplace/listing/GZTSZAS2KF7/snowflake-public-data-products-finance-economics)にあるデータを参照しているものになります。実行するためには、`modin`をパッケージに追加する必要があります。

In [ ]:
import snowflake.snowpark.modin.plugin
import modin.pandas as md
from time import perf_counter
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
# padas on Snowflakeを利用した場合
start = perf_counter()
spd_pf = md.read_snowflake("FINANCIAL__ECONOMIC_ESSENTIALS.CYBERSYN.STOCK_PRICE_TIMESERIES")
end = perf_counter()
data_size = len(spd_pf)
print(f"Snowpark Pandasに {data_size}行のデータを読み込み。処理時間 {end - start} 秒")

In [ ]:
# Daraframe APIを利用して、Pandasにデータを格納した場合
start = perf_counter()
pd_pf = session.table("FINANCIAL__ECONOMIC_ESSENTIALS.CYBERSYN.STOCK_PRICE_TIMESERIES").to_pandas()
end = perf_counter()
data_size = len(pd_pf)
print(f"pandasに {data_size}行のデータを読み込み。処理時間 {end - start} 秒")

## SQLセル内でのPython変数の使用 🔖

Jinja構文`{{..}}`を使用して、SQLクエリ内でPython変数を以下のように参照できます。

```python
threshold = 5
```

```sql
-- SQLでPython変数を参照
SELECT * FROM SNOW_CATALOG where RATING > {{threshold}}
```

これを実践して、Pythonで設定した平均値と標準偏差の値に基づいて評価の値の分布を生成してみましょう。

In [ ]:
mean = 5 
stdev = 3

In [ ]:
-- Python変数`mean`と`stdev`を使用してSQLクエリを構成する方法に注目
-- Python変数がSQLクエリを動的に構成する方法に注目
CREATE OR REPLACE TABLE SNOW_CATALOG AS 
SELECT CONCAT('SNOW-',UNIFORM(1000,9999, RANDOM())) AS PRODUCT_ID, 
       ABS(NORMAL({{mean}}, {{stdev}}, RANDOM())) AS RATING, 
       ABS(NORMAL(750, 200::FLOAT, RANDOM())) AS PRICE
FROM TABLE(GENERATOR(ROWCOUNT => 100));

In [ ]:
SELECT * FROM SNOW_CATALOG;

### サブクエリゲームのレベルアップ! 🧑‍🎓

PythonおよびSQLセルの結果参照にて学習したことと組み合わせることで、[CTEs](https://docs.snowflake.com/en/user-guide/queries-cte) で長いサブクエリを簡素化することができます。

たとえば、5を超える評価を持つすべての製品の平均評価を計算したい場合。通常、次のようなものを書く必要があります:


In [ ]:
WITH RatingsAboveFive AS (
    SELECT RATING
    FROM SNOW_CATALOG
    WHERE RATING > 5
)
SELECT AVG(RATING) AS AVG_RATING_ABOVE_FIVE
FROM RatingsAboveFive;

Snowflake Notebooksを使用すると、クエリがずっと簡単になります！Jinjaを使って別のSQLセルからSQLテーブルを参照することで、同じ結果を得ることができます。例：`{{my_cell}}`。 

In [ ]:
SELECT AVG(RATING) FROM {{variables3}}
WHERE RATING > 5

## Streamlitを利用したインタラクティブなアプリの作成🪄

これらすべてをまとめて、異なるパラメータがデータ分布ヒストグラムの形状にどのような影響を与えるかを探索するStreamlitアプリを構築してみましょう。

In [ ]:
import streamlit as st
st.markdown("# スライダーを動かして調整し、結果の更新を見てみましょう！ 👇")
col1, col2 = st.columns(2)
with col1:
    mean = st.slider('評価分布の平均値',0,10,3) 
with col2:
    stdev = st.slider('評価分布の標準偏差', 0, 10, 5)

query =f'''CREATE OR REPLACE TABLE SNOW_CATALOG AS 
SELECT CONCAT('SNOW-',UNIFORM(1000,9999, RANDOM())) AS PRODUCT_ID, 
       ABS(NORMAL({mean}, {stdev}, RANDOM())) AS RATING, 
       ABS(NORMAL(750, 200::FLOAT, RANDOM())) AS PRICE
FROM TABLE(GENERATOR(ROWCOUNT => 100));'''
session.sql(query).collect()


# Snowparkからテーブルを読み込み、結果をプロット
df = session.table("SNOW_CATALOG").to_pandas()
# Altairで結果をプロット
alt.Chart(df).mark_bar().encode(
    alt.X("RATING", bin=alt.Bin(step=2)),
    y='count()',
)

## キーボードショートカットで高速実行 🏃

これらのショートカットを使用すると、ノートブック内をより迅速にナビゲートできます。

| コマンド | ショートカット |
| --- | ----------- |
| **このセルを実行して次に進む** | SHIFT + ENTER |
| **このセルのみ実行** | CMD + ENTER |
| **すべてのセルを実行** | CMD + SHIFT + ENTER |
| **下にセルを追加** | b |
| **上にセルを追加** | a |
| **このセルを削除** | d+d |

\
右下の`?`ボタンをクリックすると、ショートカットの完全なリストを表示できます。

In [ ]:
-- チュートリアル後に環境をクリーンアップするためのクリーンアップコード
DROP TABLE SNOW_CATALOG;

# Snowflake Notebooksでファイルを扱う方法 🗄️

この例では、notebooksでファイルを扱う方法と、それらをstageに永続的に保存する方法を説明します。

## 一時ファイルの操作

notebookから書き込んだファイルは、notebookに関連付けられたlocal stageに一時的に保存されます。

**notebookセッションを終了するとすぐに、これらのファイルにアクセスできなくなることに注意してください。**

簡単なファイルを作成して、この仕組みの例を見てみましょう。

In [ ]:
import os
os.mkdir("myfolder/")
os.chdir("myfolder/")

In [ ]:
with open("myfile.txt",'w') as f:
    f.write("abc")
f.close()

stageにあるファイルを確認してみましょう。`notebook_app.ipynb`と`environment.yml`はSnowflake notebookの一部として自動的に作成されるファイルです。新しく作成したファイル`myfile.txt`が確認できます。

In [ ]:
import os
os.listdir()

では、notebookをセッションから切断してみましょう。これは、ブラウザページを閉じる/更新するか、右上の`Active`ボタンをクリックして`End session`を押すことで行えます。

このセルから開始してnotebookを再実行すると、前回のnotebookセッション中に作成したファイル`myfile.txt`は失われます。 

In [ ]:
import os
os.listdir()

## 永続ファイルの操作

セッションに戻ったときに再度アクセスできる永続的な場所にファイルを保存したい場合はどうでしょうか？例えば、モデルを訓練して後で使用するためにモデルを保存したい場合や、分析結果を保存したい場合があります。notebookセッション中に作成されたファイルはデフォルトで一時的なものなので、永続的なSnowflake stageにファイルを移動してファイルを永続的に保存する方法を説明します。

まず、`PERMANENT_STAGE`という名前のstageを作成しましょう：

In [ ]:
CREATE OR REPLACE STAGE PERMANENT_STAGE;

では、再び一時的なlocal stageに`myfile.txt`を書き込みましょう

In [ ]:
with open("myfile.txt",'w') as f:
    f.write("abc")
f.close()

では、Snowparkを使用して作成したローカルファイルをstageの場所にアップロードしましょう。Notebooksでは、`get_active_session`メソッドを使用して[session](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.Session#snowflake.snowpark.Session)コンテキスト変数を取得し、以下のようにSnowparkを操作できます：

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

Snowparkの[session.file.put](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.FileOperation.put)コマンドを使用して、`myfile.txt`をstageの場所`@PERMANENT_STAGE`に移動しましょう

In [ ]:
put_result = session.file.put("myfile.txt","@PERMANENT_STAGE", auto_compress= False)
put_result[0].status

ファイルが永続stageにアップロードされました。 

In [ ]:
LS @PERMANENT_STAGE;

notebookセッションを切断しても、ファイルが永続stageに残存していることが確認できます。

In [ ]:
LS @PERMANENT_STAGE;

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

f = session.file.get_stream("@PERMANENT_STAGE/myfile.txt")
print(f.readline())
f.close()

また、読み取る前にファイルをローカルにダウンロードしたい場合は、[session.file.get](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.FileOperation.get)コマンドを使用できます： 

In [ ]:
# Download the file from stage to current local path
get_status = session.file.get("@PERMANENT_STAGE/myfile.txt","./")
get_status[0].status

In [ ]:
import os
os.listdir()

## ボーナス: stageからのデータファイルの操作

stageは、Snowflakeに読み込まれる前にデータファイルを保存する一般的な場所です。前のセクションでは、Snowflake stageに汎用ファイルを読み書きする方法を見ました。ここでは、stageに保存されたテーブル形式のデータファイルを操作する一般的な例をいくつか紹介します。


In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

異なる日における様々なスキーリゾート地での降雪量を記録したサンプルデータセットがあります。

In [ ]:
# Create a Snowpark DataFrame with sample data
df = session.create_dataframe([[1, 'Big Bear', 8],[2, 'Big Bear', 10],[3, 'Big Bear', 5],
                               [1, 'Tahoe', 3],[2, 'Tahoe', 20],[3, 'Tahoe', 13]], 
                              schema=["DAY", "LOCATION", "SNOWFALL"])
df

Snowpark dataframeをstage上のCSVファイルに書き込む方法は次の通りです：

In [ ]:
df.write.copy_into_location("@PERMANENT_STAGE/snowfall.csv",file_format_type="csv",header=True)

stage上のファイルにアクセスするには、stageの場所からCSVファイルを読み取ってSnowpark dataframeに戻します：

In [ ]:
df = session.read.options({"infer_schema":True}).csv('@PERMANENT_STAGE/snowfall.csv')

notebooksでデータファイルを操作する方法について詳しく学ぶには、[外部S3 stageからCSVファイルを操作する方法](https://github.com/Snowflake-Labs/snowflake-demo-notebooks/blob/main/Load%20CSV%20from%20S3/Load%20CSV%20from%20S3.ipynb)と[パブリックエンドポイントからSnowflakeテーブルにデータを読み込む方法](https://github.com/Snowflake-Labs/snowflake-demo-notebooks/blob/main/Ingest%20Public%20JSON/Ingest%20Public%20JSON.ipynb)のチュートリアルをご確認ください。 

In [ ]:
-- Teardown stage created as part of this tutorial
DROP STAGE PERMANENT_STAGE;

### まとめ

このチュートリアルでは、notebookからローカルファイルを永続的なSnowflake stageにアップロードして、notebookセッション間で結果を永続化する方法を紹介しました。Snowparkのファイル操作コマンド（例：[file.get](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.FileOperation.get)、[file.put](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.FileOperation.put)）を使用して、ローカルファイルパスとstageの場所間でファイルを移動しました。Snowparkでファイルを操作することについての詳細は[こちら](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/io)で学ぶことができます。

## ノートブックの探求を続けましょう！🧭

詳細については、[サンプルノートブックギャラリー](https://github.com/Snowflake-Labs/notebook-demo)と[ドキュメント](https://docs.snowflake.com/LIMITEDACCESS/snowsight-notebooks/ui-snowsight-notebooks-about)をご確認ください！